# **Dynamixel 7-DOF robot arm (Samsung Princeternship)**

Authors: Ekin Gurgen, Ainil Norazman, Miguel Opena

This project took place from Jan 6, 2020 to Jan 10, 2020 at Samsung AI Center (New York, NY). Goal: To program a robotic arm to hook a ring. We took several approaches to this goal, and these are outlined in the coming cells. 

In [ ]:
# Take a look at:
# http://emanual.robotis.com/docs/en/software/dynamixel/dynamixel_sdk/sample_code/python_read_write_protocol_2_0/#python-protocol-20
import dynamixel_sdk
import time
import math
import warnings
import numpy as np

import realsense as rs
import apriltag
import cv2
import matplotlib.pyplot as plt

import PIL.Image
import IPython.display

import dynamixel
from dynamixel import JOINT_1, JOINT_2, JOINT_3, JOINT_4, JOINT_5, JOINT_6, JOINT_7 

## **Primitives for arm control**
These are the basic commands: initialize a control object, set the arm's position, and lock/unlock joints as desired. 

In [ ]:
JOINT_ALL = [JOINT_1, JOINT_2, JOINT_3, JOINT_4, JOINT_5, JOINT_6, JOINT_7]
JOINTS_UNLOCKED = [JOINT_1, JOINT_2, JOINT_3, JOINT_5]
JOINTS_LOCKED = [JOINT_4, JOINT_6, JOINT_7]

# JOINT_LIMITS = {1: [100,3900], 2: [1000,2738], 3: [484,3192], 5: [914, 3200]}
JOINT_LIMITS = {1: [1,3500], 2: [1005,2733], 3: [489,3187], 5: [919, 3195]}

In [ ]:
# Initializes controller object
ctl = dynamixel.DynamixelController()

In [ ]:
# Default starting position of arm
def home():
    ctl.set_position(JOINT_1, 1)
    ctl.set_position(JOINT_2, 2050)
    ctl.set_position(JOINT_3, 1850)
    ctl.set_position(JOINT_4, 3050)
    ctl.set_position(JOINT_5, 2070)
    ctl.set_position(JOINT_6, 1000)
    ctl.set_position(JOINT_7, 1200)

# Default starting position of arm (looks cooler)
def homeCool():
    ctl.set_position(JOINT_1, 1000)
    ctl.set_position(JOINT_2, 1800)
    ctl.set_position(JOINT_3, 500)
    ctl.set_position(JOINT_4, 3050)
    ctl.set_position(JOINT_5, 2400)
    ctl.set_position(JOINT_6, 1050)
    ctl.set_position(JOINT_7, 1200)

home()

In [ ]:
# Get position of all joints
for joint in JOINT_ALL: 
    print(ctl.get_position(joint)) 

In [ ]:
# Get position of specific joint
print(ctl.get_position(JOINT_1))

In [ ]:
# Unlock joints listed in JOINTS_UNLOCKED
for joint in JOINTS_UNLOCKED:
    ctl.unlock(joint)

In [ ]:
# Unlock a specific joint
ctl.unlock(JOINT_5)

In [ ]:
# Lock a specific joint
ctl.lock(JOINT_1)

In [ ]:
# Unlock all joints
for joint in JOINT_ALL:
    ctl.unlock(joint)

In [ ]:
# Lock all joints
for joint in JOINT_ALL:
    ctl.lock(joint)

In [ ]:
# Deletes the control object and unlocks all joints
del ctl

## **Control Strategy No.1:** Trajectory mimicry
The arm replays a trajectory based on how a human user guides it. 

Currently, the movement is jagged and slow. We could add velocity control to make it smoother. 

In [ ]:
''' Returns a dictionary of poses over time (user guides the arm) '''
def trajectoryCopy(num_iterations=50, delay=0.1): 
    traj = {}
    for i in range(num_iterations):
        print(f"Iteration {i}")
        trajNow = {}
        for joint in JOINTS_UNLOCKED:
            pos = ctl.get_position(joint)
            trajNow[joint] = pos
        traj[i] = trajNow
        time.sleep(delay)
    return traj

''' Iterates through a dictionary of poses and replays them with a slowdown factor '''
def trajectoryMimic(num_iterations=50, delay=0.1, slowdown=4): 
    for i in range(num_iterations):
        location = traj[i]
        for joint in location:
            ctl.set_position(joint, location[joint])
        time.sleep(delay * slowdown)

In [ ]:
# Test code for trajectory mimicry
# Note: unlock the arm to use this feature
delay = 0.2
num_iterations = 40
traj = trajectoryCopy(num_iterations, delay)
for i in range(num_iterations):
    print(traj[i])
trajectoryMimic(num_iterations, delay, slowdown=2)

## **Control Strategy No.2:** Inverse kinematics

Implementing inverse kinematics for a 4-DOF model of the robotic arm. We are only using Joints 1, 2, 3 and 5; the other joints remain locked. 

To simplify things, we can treat the arm as a 3-DOF manipulator in the plane (Joints 2, 3, and 5). Then, we can have Joint 1 rotate this 3-DOF arm radially, so that it can rotate in 3D space.

Suppose we have a 3-DOF model of a robotic arm with three segments. It is constrained to move in a plane. Let $(x,y)$ denote the end effector's position in 2D space. Let $\phi$ denote the angle of said effector. We can describe the length of each segment as $(l_1,l_2,l_3)$ and the angle of each joint actuator as $(\theta_1,\theta_2,\theta_3)$. Then, the direct kinematics of the system are

\begin{align}
    x &= l_1 \cos\theta_1 + l_2 \cos(\theta_1 + \theta_2) + l_3 \cos(\theta_1 + \theta_2 + \theta_3)\\
    x &= l_1 \sin\theta_1 + l_2 \sin(\theta_1 + \theta_2) + l_3 \sin(\theta_1 + \theta_2 + \theta_3)\\
    \phi &= \theta_1 + \theta_2 + \theta_3
\end{align}

For inverse kinematics, we start with $(x,y,\theta)$ and arrive at $(\theta_1,\theta_2,\theta_3)$. Due to the nature of the solution, we have to consider a parameter $\sigma$. When we set $\sigma=+1$, the robot moves into an ''elbow-up'' position. When we set $\sigma=-1$, the robot moves into an ''elbow-down'' position. I do not want to type out all the equations. Refer to page 19-20 of the source below. 

Source: notes from [Vijay Kumar](https://www.seas.upenn.edu/~meam520/notes02/IntroRobotKinematics5.pdf)

In [ ]:
''' Variables for arm lengths '''
h = 0.054 
l1 = 0.170
l2 = 0.115
l3 = 0.23

''' Converts Dynamixel motor position (0-4000) to radians (0-2pi) '''
def toRad(position):
    return (position-2000) * (math.pi/2000)

''' Converts radians (0-2pi) to Dynamixel motor position (0-4000) '''
def toPosition(angle):
    return (angle*2000/math.pi) + 2000

''' Executes forward kinematics for the Dynamixel robotic arm '''
def forwardKinematics(theta1, theta2, theta3):
    xcurrent = l1 * math.cos(theta1) + l2 * math.cos(theta1+theta2) + l3 * math.cos(theta1+theta2+theta3)            
    ycurrent = l1 * math.sin(theta1) + l2 * math.sin(theta1+theta2) + l3 * math.sin(theta1+theta2+theta3)
    return xcurrent, ycurrent

''' Executes inverse kinematics for the Dynamixel robotic arm'''
def inverseKinematics(xGoal, yGoal, phiGoal, sigma=1):
    
    x_prime = xGoal - l3*math.cos(phiGoal)
    y_prime = yGoal - l3*math.sin(phiGoal)
    A = x_prime**2 + y_prime**2
    
    gamma = math.atan2(-y_prime/math.sqrt(A), x_prime/math.sqrt(A))
    
    cosarg = -(A + l1**2 - l2**2)/(2*l1*math.sqrt(A))
    if cosarg < -1: cosarg = -1
    if cosarg > 1: cosarg = 1
    theta1 = gamma + sigma*math.acos(cosarg)
    theta1 = math.pi - theta1
    theta2 = math.atan2((y_prime - l1*math.sin(theta1))/l2, (x_prime - l1*math.cos(theta1))/l2) - theta1
    theta3 = phiGoal - theta1 - theta2

    position1 = int(-toPosition(theta1 - math.pi/2) % 4000) # % 4000
    position2 = int(toPosition(theta2) - 150) % 4000
    position3 = int(toPosition(theta3)) % 4000
    

    if not JOINT_LIMITS[2][0] <= position1 <= JOINT_LIMITS[2][1]: 
        raise ValueError(f'Joint 2 position {position1} exceeds motor limits.')
    if not JOINT_LIMITS[3][0] <= position2 <= JOINT_LIMITS[3][1]: 
        raise ValueError(f'Joint 3 position {position2} exceeds motor limits.')
    if not JOINT_LIMITS[5][0] <= position3 <= JOINT_LIMITS[5][1]: 
        raise ValueError(f'Joint 5 position {position3} exceeds motor limits.')
    
    return theta1, theta2, theta3, position1, position2, position3

### Rotating the base joint

If we model Joints 2, 3, and 5 as a planar 3-DOF manipulator, we can simply rotate Joint 1 to give the robot access to many more places in 3D space. Calculating where Joint 1 should rotate is a simple matter of trigonometry, once one accounts for the restricted domain of the arctan function. 

In [ ]:
''' Rotates Joint 1 (base) of the robotic arm '''
def getBasePosition(xGoal, yGoal, zGoal):
    
    offset = 8 * math.pi / 180 # angle offset to account for hook placement
    
    if xGoal == 0 and zGoal >= 0: 
        zeta = math.pi / 2
    elif xGoal == 0 and zGoal < 0:
        zeta = -math.pi / 2
    else: zeta = math.atan(zGoal / xGoal)
    
    zeta += offset
    
    if xGoal < 0: zeta += math.pi
        
    # Converts angle to Dynamixel motor position
    pos = int(toPosition(zeta) - 2000) % 4000
    
    if not JOINT_LIMITS[1][0] <= pos <= JOINT_LIMITS[1][1]: 
        raise ValueError(f'Joint 1 position {pos} exceeds motor limits.')
    return zeta, pos

### Hooking the ring
A small subroutine to hook a ring. We used a bit of trial-and-error to establish the overall motion (inch forward into the ring and then pull up), as well as some fine-tuned parameters. 

In [ ]:
''' Code for hooking a ring '''
def scoop(push_constant=100, pull_constant=800):
    pos2 = ctl.get_position(JOINT_2)
    pos3 = ctl.get_position(JOINT_3)
    pos5 = ctl.get_position(JOINT_5)
    
    # Push forward into ring
    for i in range(3):
        ctl.set_position(JOINT_3, pos3 + (i+1)*push_constant)
        ctl.set_position(JOINT_2, pos2 + (i+1)*push_constant)
        ctl.set_position(JOINT_5, pos5 + (i+1)*push_constant // 2)
        time.sleep(1)
    
    # Pulls ring upward and retracts arm
    ctl.set_position(JOINT_5, pos5 + pull_constant)
    time.sleep(1)
    ctl.set_position(JOINT_2, pos2)
    ctl.set_position(JOINT_3, pos3)

### Full control routine (inverse kinematics only)
Given a position in 3D XYZ coordinates, the robot can navigate to said position (as long as it is reachable) with centimeter accuracy. 

In [ ]:
''' Code for executing the full arm routine '''
def fullRoutine(goalPose, phiGoal, sigma=1, scoopParams=(70,500), doScoop=True):
    # Retrieve coordinates
    xGoal = goalPose[0]
    yGoal = goalPose[1]
    zGoal = goalPose[2]
    
    # Home robot to pre-defined start position
    home()
    time.sleep(1)

    # Get data from inverse kinematics
    flatDistance = math.sqrt(xGoal**2 + zGoal**2)
    p1, p2, p3, position1, position2, position3 = inverseKinematics(flatDistance, yGoal, phiGoal, sigma)
    zeta, position0 = getBasePosition(xGoal, yGoal, zGoal)

    # Setting position of robot base from getBasePosition
    ctl.set_position(JOINT_1, position0)
    time.sleep(1)

    # Setting position of robot arm from inverse kinematics
    ctl.set_position(JOINT_2, position1)
    ctl.set_position(JOINT_3, position2)
    ctl.set_position(JOINT_5, position3)
    time.sleep(1)
    
    # Setting position of robot arm from scooping subroutine
    if doScoop: 
        scoop(scoopParams[0], scoopParams[1])

### Miscellaneous testing code
We used this code to test inverse kinematics, making sure that inverse and forward kinematics are inverse functions of each other. 

In [ ]:
''' Code for testing the forward and inverse kinematics '''
theta1 = 70*math.pi/180
# theta2 < 0, theta3 > 0 <--> sigma = 1
# theta2 > 0, theta3 < 0 <--> sigma = -1
theta2 = -30*math.pi/180
theta3 = 20*math.pi/180
phiGoal = theta1 + theta2 + theta3
print(theta1)
print(theta2)
print(theta3)

xGoal, yGoal = forwardKinematics(theta1, theta2, theta3)
print(xGoal, yGoal)
p1, p2, p3, position1, position2, position3 = inverseKinematics(xGoal, yGoal, phiGoal)
print(p1, p2, p3)
print(p1 * 180 / math.pi, p2 * 180 / math.pi, p3 * 180 / math.pi)
print(forwardKinematics(p1, p2, p3))

print(position1)
print(position2)
print(position3)

## **Control Strategy No.3:** Vision (April tags)

We attached an Intel Realsense D435 camera to the computer, mounting said camera on a tripod overlooking our test space. It locates the arm and the target using April tags. From these locations, we calculate the position in 3D space that the arm should navigate towards, then we use inverse kinematics to travel to the desired location. 

In [ ]:
# Camera parameters w.r.t. camera image
fx = 607.91638184
fy = 609.396240230
cx = 322.27319336
cy = 240.48545837
cam_params = [fx, fy, cx, cy]

# Distortion parameters
# Source: https://github.com/IntelRealSense/librealsense/issues/573
coeffs = [-0.1494821, 0.01267229, -0.002873175, -0.002943313, -0.02257685]

# Various offsets for arm
ARM_HEIGHT = 0.385
HOOK_LENGTH = 0.13
RING_RADIUS = 0.07

# Conversion (because America)
INCH_TO_MM = 25.4
INCH_TO_METERS = INCH_TO_MM / 1000

# Size of the provided April tags
tag_size = 33.4 / INCH_TO_MM

In [ ]:
''' Function to initialize camera object that reads from Realsense '''
def initCamera(): 
    serials = rs.RealSenseReal.connected_device_serial_numbers()

    cam = rs.RealSenseReal(
        serial_number=serials[0],
        depth_sensor_preset=rs.RealSenseD400VisualPreset.DEFAULT,
        temporal_filter_parameters=rs.TemporalFilterParameters(),
        ignore_depth_scale_offset_calibration=True,
    )
    
    return cam

In [ ]:
''' Function to display images from camera '''
def displayImage(image):
    plt.imshow(image)
    plt.show()

In [321]:
''' Function to display images from camera (uses IPython display) '''
def showarray(a):
    IPython.display.display(PIL.Image.fromarray(a[:, :, ::-1]))

In [ ]:
''' Code for testing the full arm routine with VISION (April tags) '''
''' Ingredients: Dynamixel 7-DOF robotic arm, April tag on top of arm, different April tag on top of tape roll '''
def fullRoutineVisionApril(cam, phiGoal, sigma=1, scoopParams=(70,500), disp=False):
    # Home robot to pre-defined start position
    home()
    time.sleep(2)
    
    # Capture image from camera object
    image = cam.gray(norm=False)
    if disp: displayImage(image)
    # print(image)cam.gray(norm=False)
    
    # Detect the April tags in image
    detector = apriltag.Detector()
    detections, dimg = detector.detect(image, return_image=True)
    result = detector.detect(image)

    # check if tags are detected
    if len(result) < 2: raise ValueError(f"Error: missing tag. Recognized tag: {result[0].tag_id}")
    elif len(result) > 2: raise ValueError("too many tags in the image")
        
    # Assumption: the arm has tag ID smaller than the object tag ID
    arm = result[0]
    # Refers to center of the arm tag w.r.t. camera image
    arm_xPixel = int(round(arm.center[0]))
    arm_yPixel = int(round(arm.center[1]))
    
    obj = result[1]
    # Refers to center of the object tag w.r.t. camera image
    obj_xPixel = int(round(obj.center[0]))
    obj_yPixel = int(round(obj.center[1]))
    
    # Detect camera-frame poses of arm and object (rotation-translation matrices)
    pose_arm, e0, e1 = detector.detection_pose(arm, cam_params, tag_size)
    pose_obj, e2, e3 = detector.detection_pose(obj, cam_params, tag_size)
    
    # Convert poses to meters
    arm_x = pose_arm[0][3] * INCH_TO_METERS
    arm_y = pose_arm[1][3] * INCH_TO_METERS
    arm_z = pose_arm[2][3] * INCH_TO_METERS

    obj_x = pose_obj[0][3] * INCH_TO_METERS
    obj_y = pose_obj[1][3] * INCH_TO_METERS
    obj_z = pose_obj[2][3] * INCH_TO_METERS

    # Gets camera-frame pose data as XYZ coordinates
    arm_loc = np.array([arm_x, arm_y, arm_z])
    obj_loc = np.array([obj_x, obj_y, obj_z])
    
    # Computes difference between arm and object locations
    diff = obj_loc - arm_loc
    diff = np.append(diff, [1])
    
    # Assumption: the April tag on the ARM lies flat with respect to the table surface
    # Account for coordinate transformation: camera --> top of arm
    rotation_matrix = np.array(pose_arm)
    rotation_matrix[0][3] = 0
    rotation_matrix[1][3] = 0
    rotation_matrix[2][3] = 0
    
    # Uses formula from http://ogldev.atspace.co.uk/www/tutorial13/tutorial13.html
    rot_inv = np.linalg.inv(rotation_matrix)
    diff_real = np.matmul(rot_inv, diff)
    
    # Convert axes to our convention
    diff_real_adjusted = np.array([diff_real[1], diff_real[2], diff_real[0]])
    
    # Account for vertical offset; ring radius
    # Account for vector change: origin of arm --> center of ring
    diff_real_adjusted[1] = ARM_HEIGHT - (diff_real_adjusted[1] + RING_RADIUS) + HOOK_LENGTH
    diff_real_adjusted[0] = diff_real_adjusted[0] * 0.9
    diff_real_adjusted[1] = diff_real_adjusted[1] * 0.8
    diff_real_adjusted[2] = diff_real_adjusted[2] * 0.9
    
    fullRoutine(diff_real_adjusted, phiGoal, sigma=sigma, scoopParams=scoopParams)
    
    return diff_real_adjusted

In [ ]:
cam = initCamera()

In [ ]:
phiGoal = -5 * math.pi / 180 # try to make phi positive if possible
fullRoutineVisionApril(cam, phiGoal, disp=True)

## **Control Strategy No.4**: Vision (color detection)

We attached an Intel Realsense D435 camera to the computer, mounting said camera on a tripod overlooking our test space. It locates the arm using an April tag, but it uses color detection and a bounding box to locate a blue ring (roll of tape), which it targets and attempts to hook using our previous code on inverse kinematics.

In [ ]:
# Formula to deproject from 2D image space to 3D camera space
x = (centerx - cx) / fx
y = (centery - cy) / fy
point = [pixelDepth * x, pixelDepth * y, pixelDepth]

In [ ]:
''' Code for testing the full arm routine with VISION (color) '''
''' Ingredients: Dynamixel 7-DOF robotic arm, blue tape roll, April tag on top of arm '''
def fullRoutineVisionColor(cam, phiGoal, sigma=1, scoopParams=(70,500), disp=False, lowerThreshold=250, ratio=3):
    # Home robot to pre-defined start position
    home()
    time.sleep(2)

    # Capture image from camera object
    image = cam.gray(norm=False)
    imageColor = cam.color(norm=False)
    imageDepth = cam.depth(norm=False)
    if disp: 
        displayImage(image)
        displayImage(imageDepth)
        # showarray(image)
    # print(image)cam.gray(norm=False)

    ''' Determine pose of robotic arm '''
    # Detect the April tag in image (robotic arm only)
    detector = apriltag.Detector()
    detections, dimg = detector.detect(image, return_image=True)
    result = detector.detect(image)

    # check if tags are detected
    if len(result) < 1: raise ValueError(f"Error: missing tag on the robotic arm.")
    elif len(result) > 2: raise ValueError("too many tags in the image")

    # Assumption: the arm has tag ID smaller than the object tag ID
    arm = result[0]
    # Refers to center of the arm tag w.r.t. camera image
    arm_xPixel = int(round(arm.center[0]))
    arm_yPixel = int(round(arm.center[1]))

    # Detect camera-frame poses of arm and object (rotation-translation matrices)
    pose_arm, e0, e1 = detector.detection_pose(arm, cam_params, tag_size)

    # Convert poses to meters
    arm_x = pose_arm[0][3] * INCH_TO_METERS
    arm_y = pose_arm[1][3] * INCH_TO_METERS
    arm_z = pose_arm[2][3] * INCH_TO_METERS
    
    ''' Determine pose of ring '''
    # Identify range of blues
    lower = np.array([70, 0, 0], dtype="uint8")
    upper = np.array([255, 210, 50], dtype="uint8")

    # Apply a mask based on range of blues
    mask = cv2.inRange(imageColor, lower, upper)
    output = cv2.bitwise_and(imageColor, imageColor, mask=mask)

    # Blur out some noise
    output = cv2.medianBlur(output, 5)
    
    # Define the contours in the image to find the ring
    edged = cv2.Canny(output, lowerThreshold, lowerThreshold * ratio)
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    output = cv2.drawContours(output, contours, -1, (0, 255, 0), 3)
    # if disp: print(contours)

    # Produce a bounding box around the ring
    maxCnt = None
    maxBox = None
    maxArea = 0
    for cnt in contours: 
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        area = cv2.contourArea(cnt)
        if area > maxArea:
            maxCnt = cnt
            maxArea = area
            maxBox = box

    output = cv2.drawContours(output, [maxBox], 0, (0, 0, 255), 2)
    # print(maxBox)
    if disp: showarray(output)
        
    # Source: https://docs.opencv.org/3.4/dd/d49/tutorial_py_contour_features.html
    # centerx, centery refer to top of the bounding box
    centerx = (maxBox[0][0] +maxBox[1][0] + maxBox[2][0] + maxBox[3][0]) // 4
    centery = (maxBox[0][1] +maxBox[1][1] + maxBox[2][1] + maxBox[3][1]) // 4
    pixelLoc = [centerx, centery]
    pixelDepth = imageDepth[centery][centerx] / 1000
    pixelDepth = pixelDepth * 1.01
    
    # Warn the user if pixelDepth is zero
    if pixelDepth < 1e-2:
        warnings.warn("Pixel depth registered as zero. Arm will not work.")
        
    # Formula to deproject from 2D image space to 3D camera space
    x = (centerx - cx) / fx
    y = (centery - cy) / fy
    point = [pixelDepth * x, pixelDepth * y, pixelDepth]
    if disp: print(point)

    ''' Consolidate arm and ring data into the same frame '''
    # Gets camera-frame pose data as XYZ coordinates
    arm_loc = np.array([arm_x, arm_y, arm_z])
    obj_loc = point

    # Computes difference between arm and object locations
    diff = obj_loc - arm_loc
    diff = np.append(diff, [1])

    # Assumption: the April tag on the ARM lies flat with respect to the table surface
    # Account for coordinate transformation: camera --> top of arm
    rotation_matrix = np.array(pose_arm)
    rotation_matrix[0][3] = 0
    rotation_matrix[1][3] = 0
    rotation_matrix[2][3] = 0

    # Uses formula from http://ogldev.atspace.co.uk/www/tutorial13/tutorial13.html
    rot_inv = np.linalg.inv(rotation_matrix)
    diff_real = np.matmul(rot_inv, diff)

    # Convert axes to our convention
    diff_real_adjusted = np.array([diff_real[1], diff_real[2], diff_real[0]])

    # Account for vertical offset; ring radius
    # Account for vector change: origin of arm --> center of ring
    diff_real_adjusted[1] = ARM_HEIGHT - diff_real_adjusted[1] + HOOK_LENGTH - 0.1
    diff_real_adjusted[0] = diff_real_adjusted[0] * 0.9
    diff_real_adjusted[1] = diff_real_adjusted[1] * 0.8
    diff_real_adjusted[2] = diff_real_adjusted[2] * 0.9

    fullRoutine(diff_real_adjusted, phiGoal, sigma=sigma, scoopParams=scoopParams, doScoop=True)

In [ ]:
home()

In [ ]:
cam = initCamera()

In [ ]:
phiGoal = -10 * math.pi / 180 # try to make phi positive if possible
fullRoutineVisionColor(cam, phiGoal, disp=True)